# Content Engagement Analytics - Data Exploration

This notebook explores the engagement analytics data from our PostgreSQL database, providing insights into author performance, engagement patterns, and content strategy opportunities.

## Table of Contents
1. [Database Connection](#database-connection)
2. [Top Authors Analysis](#top-authors-analysis)
3. [Engagement Patterns Heatmap](#engagement-patterns-heatmap)
4. [Author Performance Segmentation](#author-performance-segmentation)
5. [Business Insights & Recommendations](#business-insights)

## 1. Database Connection

Connect to PostgreSQL and load the SQL queries.

In [ ]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully")

In [ ]:
# Database connection
conn = psycopg2.connect(
    host="127.0.0.1",
    port=13177,
    database="engagement_db",
    user="analytics",
    password="analytics_pass"
)

print("Connected to PostgreSQL successfully!")
print(f"PostgreSQL version: {conn.server_version}")

## 2. Top Authors Analysis

Identify top-performing authors across categories.

In [ ]:
# Load top authors data
with open('../sql/top_authors.sql', 'r') as f:
    top_authors_query = f.read()

df_top_authors = pd.read_sql_query(top_authors_query, conn)
print(f"Loaded {len(df_top_authors)} author-category combinations")
print(f"\nTop 10 performers:")
df_top_authors.head(10)

In [ ]:
# Visualize top authors by total engagement
top_20 = df_top_authors.nlargest(20, 'total_engagements')

fig = px.bar(
    top_20,
    x='author_name',
    y='total_engagements',
    color='author_category',
    title='Top 20 Authors by Total Engagement',
    labels={'total_engagements': 'Total Engagements', 'author_name': 'Author'},
    height=500
)
fig.update_xaxis(tickangle=-45)
fig.show()

## 3. Engagement Patterns Heatmap

Analyze when users are most engaged (hour of day × day of week).

In [ ]:
# Load engagement patterns
with open('../sql/engagement_patterns.sql', 'r') as f:
    patterns_query = f.read()

df_patterns = pd.read_sql_query(patterns_query, conn)
print(f"Loaded {len(df_patterns)} hour-day combinations")
print(f"\nSample data:")
df_patterns.head()

In [ ]:
# Create engagement heatmap
pivot_data = df_patterns.pivot(
    index='hour_of_day',
    columns='day_name',
    values='total_engagements'
)

# Reorder columns to start with Monday
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
pivot_data = pivot_data[[col for col in day_order if col in pivot_data.columns]]

plt.figure(figsize=(14, 10))
sns.heatmap(
    pivot_data,
    annot=True,
    fmt='.0f',
    cmap='YlOrRd',
    cbar_kws={'label': 'Total Engagements'},
    linewidths=0.5
)
plt.title('Engagement Heatmap: Hour of Day vs Day of Week', fontsize=16, fontweight='bold')
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Hour of Day', fontsize=12)
plt.tight_layout()
plt.show()

print("\nKey Insights:")
print(f"Peak hour: {pivot_data.max(axis=1).idxmax()}:00")
print(f"Peak day: {pivot_data.sum().idxmax()}")
print(f"Lowest engagement: {pivot_data.sum().idxmin()}")

In [ ]:
# Interactive heatmap with Plotly
fig = go.Figure(data=go.Heatmap(
    z=pivot_data.values,
    x=pivot_data.columns,
    y=pivot_data.index,
    colorscale='YlOrRd',
    hoverongaps=False,
    colorbar=dict(title="Engagements")
))

fig.update_layout(
    title='Interactive Engagement Heatmap',
    xaxis_title='Day of Week',
    yaxis_title='Hour of Day',
    height=600
)
fig.show()

## 4. Author Performance Segmentation

Identify coaching opportunities: authors with high volume but low engagement.

In [ ]:
# Load volume vs engagement data
with open('../sql/volume_vs_engagement.sql', 'r') as f:
    volume_query = f.read()

df_volume = pd.read_sql_query(volume_query, conn)
print(f"Loaded {len(df_volume)} authors")
print(f"\nPerformance segments:")
print(df_volume['performance_segment'].value_counts())

In [ ]:
# Scatter plot: Volume vs Quality
fig = px.scatter(
    df_volume,
    x='total_posts',
    y='avg_engagement_per_post',
    size='opportunity_score',
    color='performance_segment',
    hover_data=['author_name', 'author_category'],
    title='Author Performance: Volume vs Engagement Quality',
    labels={
        'total_posts': 'Total Posts (Volume)',
        'avg_engagement_per_post': 'Avg Engagement per Post',
        'opportunity_score': 'Opportunity Score'
    },
    height=600
)

fig.add_hline(
    y=df_volume['category_median'].median(),
    line_dash="dash",
    line_color="red",
    annotation_text="Median Engagement"
)

fig.show()

In [ ]:
# Top opportunity authors (highest coaching potential)
high_opportunity = df_volume.nlargest(15, 'opportunity_score')

fig = go.Figure()

fig.add_trace(go.Bar(
    x=high_opportunity['author_name'],
    y=high_opportunity['opportunity_score'],
    marker_color='indianred',
    name='Opportunity Score',
    text=high_opportunity['opportunity_score'],
    textposition='outside'
))

fig.update_layout(
    title='Top 15 Authors by Coaching Opportunity Score',
    xaxis_title='Author',
    yaxis_title='Opportunity Score',
    height=500,
    showlegend=False
)

fig.update_xaxis(tickangle=-45)
fig.show()

print(f"\nTotal opportunity score (top 15): {high_opportunity['opportunity_score'].sum():.0f}")
print("Coaching these authors to category median would add this many engagements to the platform.")

## 5. Business Insights & Recommendations

Summary of actionable insights from the data analysis.

In [ ]:
# Calculate key metrics
total_authors = len(df_volume)
high_vol_low_eng = len(df_volume[df_volume['performance_segment'] == 'High Volume, Low Engagement'])
total_opportunity = df_volume['opportunity_score'].sum()
avg_posts = df_volume['total_posts'].mean()

# Peak engagement times
peak_hour = pivot_data.max(axis=1).idxmax()
peak_day = pivot_data.sum().idxmax()
weekday_avg = pivot_data[[d for d in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] if d in pivot_data.columns]].sum().sum()
weekend_avg = pivot_data[[d for d in ['Saturday', 'Sunday'] if d in pivot_data.columns]].sum().sum()

print("="*60)
print("BUSINESS INSIGHTS & RECOMMENDATIONS")
print("="*60)

print(f"\n1. AUTHOR PERFORMANCE")
print(f"   - Total active authors: {total_authors}")
print(f"   - High-volume, low-engagement: {high_vol_low_eng} ({high_vol_low_eng/total_authors*100:.1f}%)")
print(f"   - Total coaching opportunity: {total_opportunity:.0f} engagements")
print(f"   - Potential platform impact: {total_opportunity/df_volume['total_engagements'].sum()*100:.1f}% increase")

print(f"\n2. OPTIMAL POSTING TIMES")
print(f"   - Peak hour: {peak_hour}:00")
print(f"   - Peak day: {peak_day}")
print(f"   - Weekday vs Weekend: {weekday_avg/weekend_avg:.1f}x higher engagement")

print(f"\n3. RECOMMENDED ACTIONS")
print(f"   a. Coach top {min(10, high_vol_low_eng)} high-opportunity authors")
print(f"   b. Schedule important content for {peak_day} mornings")
print(f"   c. Implement intelligent post scheduling (9am-2pm weekdays)")
print(f"   d. A/B test content formats for underperformers")

print("\n" + "="*60)

In [ ]:
# Export summary statistics
summary_stats = pd.DataFrame({
    'Metric': [
        'Total Authors',
        'Avg Posts per Author',
        'High-Opportunity Authors',
        'Total Opportunity Score',
        'Peak Engagement Hour',
        'Peak Engagement Day',
        'Weekday vs Weekend Multiplier'
    ],
    'Value': [
        total_authors,
        f"{avg_posts:.1f}",
        high_vol_low_eng,
        f"{total_opportunity:.0f}",
        f"{peak_hour}:00",
        peak_day,
        f"{weekday_avg/weekend_avg:.2f}x"
    ]
})

print("\nSummary Statistics:")
summary_stats

In [ ]:
# Close database connection
conn.close()
print("Database connection closed.")